# New and improved Parflow IO routines, integration with `xarray`, and some nifty tools for you non-Parflowheads

This notebook details some of the new Parflow IO routines in the Parflow python package. As of Jan 27, 2022 they are not quite in the main repo, so can't be `pip` installed yet. But that should happen soon - for those interested here's the [pull request](https://github.com/parflow/parflow/pull/365). This tutorial can also be run via binder through [this link](https://mybinder.org/v2/gh/arbennett/group_meeting_feb1/HEAD), alternatively here is [the GitHub repo](https://github.com/arbennett/group_meeting_feb1).

## First up - why?

I first undertook this rewrite because I found some flaws in the existing workflow for the Parflow IO routines. First, there was an underlying memory leak because of the way that the ParflowIO package holds a copy of data in the background and you interact with the data by copying it (or pieces of it). Second, it was very slow to pull out timeseries because of the way that ParflowIO reads the file header and subgrid headers every time you open a file, which ends up being a rather large overhead when you are reading thousands of files. This new method addresses both of these problems, as well as offers some other nice benefits. It is written in pure python so it is easier to understand and modify. I have also written a backend to the [xarray](https://xarray.pydata.org/en/stable/) python package, which is massively useful for data analysis.

## What you will get out of this tutorial/notebook/demo/thing

Okay, so that was the motivation for me to do all of this work, but what's in it for you? By putting together this notebook I hope to convince you that the new tools offer a compelling and performant way to analyze Parflow output. For those not really into Parflow specifically, I will show you some cool things like:

 * Loading data over the network from Google drive via [gdrivefs](https://github.com/fsspec/gdrivefs) (And more generally [fsspec](https://filesystem-spec.readthedocs.io/en/latest/?badge=latest#))
 * How xarray and the larger [Pangeo ecosystem](https://pangeo.io/) can make your data-analysis-life much easier and intuitive
 
Let's get to it - as always, starting with some imports and fixing up the plotting style.

In [ ]:
import numpy as np
import xarray as xr
import parflow as pf
import gdrivefs
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set a bigger default plot size
mpl.rcParams['figure.figsize'] = (12, 8)
mpl.rcParams['font.size'] = 22

## Intro to `gdrivefs`, or how to interact with Google drive via the notebook

Because `git` doesn't like to have large datasets put into the repo I had to get a bit creative in making this tutorial open and accessible. There's this awesome library [fsspec](https://filesystem-spec.readthedocs.io/en/latest/?badge=latest#) that I mentioned earlier, which basically lets you access different filesystems directly in python. I've got a whole [(unfinished) blog post](http://arbennett.github.io/software,/machine-learning/2021/08/18/ml_infrastructure_for_eess_beginners.html) on using fsspec - but wanted to use Google Drive here because we have unlimited storage at UA. Enter `gdrivefs`, which is basically an implementation of `fsspec` on top of Google Drive. It's pretty bare-bones but works pretty well in my experience.

To get started, you can run the cell below which points to my files for the rest of the tutorial. When starting up the `gdrivefs` session you'll have to enter a token and allow access from the link provided. Once you have granted access you'll get a code that you should copy and paste into the box that appears below the cell. With that you'll have access to the files.

In [ ]:
token = 'browser'
data_id = '1DMXwWqrF3-vhdYZcUU-RAYFYQv6BuL-T'
gdfs = gdrivefs.GoogleDriveFileSystem(token=token, root_file_id=data_id)
gdfs

### Looking at the data

With the connection to Google drive open you can interact with some Unix-like interfaces. First, let's just poke around and see the files using `ls` which just lists files and directories. You will see there are two directories here, `conus1_data` and `taylor_data`. Let's also see what's in the `conus1_data` directory by using `ls("conus1_data")` - you should see a bunch of files that we will analyze here.

In [ ]:
gdfs.ls("")

In [ ]:
gdfs.ls("conus1_data")

### Getting the data

In [ ]:
filename = 'CONUS.2004.out.press.00000.pfb'
gdfs.download(f'/conus1_data/{filename}', 
              f'{filename}')

In [ ]:
pressure = pf.read_pfb(filename)
print(pressure.shape)

In [ ]:
plt.imshow(np.mean(pressure, axis=0))
plt.colorbar(shrink=0.7)